# 練習內容
運用 NMS 來過濾預測框

# 練習目的
了解 NMS 的運作
了解為什麼需要 NMS 來完成最終的輸出結果

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def nms(bounding_boxes, confidence_score, threshold):
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []
   
    boxes = np.array(bounding_boxes)

    # 取出每一個 BOX 的 x1, y1, x2, y2
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Confidence scores of bounding boxes
    score = np.array(confidence_score)

    # 預設 list，用來保存 Bounding Box
    picked_boxes = []
    picked_score = []

    # 計算每一個 Box 的面積
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)
    
    # 排列 Box 的分數（由小至大的 indices）
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])

        # Compute ordinates of intersection-over-union(IoU)
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0, x2 - x1 + 1)
        h = np.maximum(0, y2 - y1 + 1)
        intersection = w * h

        # 計算 IoU
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)
        
        # 保存重疊率小於 threshold 的 Boxes
        less_than_threshold = np.where(ratio < threshold)
        order = order[less_than_threshold]

    return picked_boxes, picked_score

參數設置

In [ ]:
# Image path
image_path = 'image/nms.jpg'

# Bounding boxes
bounding_boxes = [(200, 82, 350, 317), (180, 67, 305, 282), (200, 90, 378, 304)]
confidence_score = [0.9, 0.75, 0.8]

# Read image
image = cv2.imread(image_path)

# Copy image as original
orginial_image = image.copy()

# Draw parameters
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

# 閾值
threshold = 0.5

經 NMS 前

In [ ]:
# Draw bounding boxes and confidence score
for (start_x, start_y, end_x, end_y), confidence in zip(bounding_boxes, confidence_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(orginial_image, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(orginial_image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(orginial_image, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# revert BGR to RGB
plt.imshow(org[:,:,::-1])
plt.show()

經 NMS 後

In [ ]:
# Run non-max suppression algorithm
picked_boxes, picked_score = nms(bounding_boxes, confidence_score, threshold)

# Draw bounding boxes and confidence score after non-maximum supression
for (start_x, start_y, end_x, end_y), confidence in zip(picked_boxes, picked_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(image, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(image, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# revrt BGR to RGB
plt.imshow(image[:,:,::-1])
plt.show()

## 比較

In [ ]:
# Show image
plt.imshow(org[:,:,::-1])
plt.show()

plt.imshow(image[:,:,::-1])
plt.show()